## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [6]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")



In [7]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b",groq_api_key=groq_api_key)

In [8]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is VaibhaV and I am a Chief AI Engineer")])

AIMessage(content='Hello VaibhaV! It’s great to meet you. As a Chief AI Engineer, you must be working on some fascinating projects. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says "Hi , My name is VaibhaV and I am a Chief AI Engineer". Likely they want a response, maybe a greeting. No request for disallowed content. So respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 87, 'total_tokens': 175, 'completion_time': 0.182214664, 'completion_tokens_details': {'reasoning_tokens': 44}, 'prompt_time': 0.003211255, 'prompt_tokens_details': None, 'queue_time': 0.047366075, 'total_time': 0.185425919}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_1d1727abc9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f034be18-73b9-4de7-b382-b6f614cb9ee6-0', usage_metadata={'input_tokens': 87, 'output_tokens': 88, 'total_tokens': 175, 'output_toke

In [9]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi do you know who I am?")])

AIMessage(content='I don’t have any information about you unless you share it with me. Feel free to tell me a bit about yourself or let me know how I can help!', additional_kwargs={'reasoning_content': 'The user asks: "Hi do you know who I am?" There\'s no context. According to policy, we can respond politely, ask for clarification. No disallowed content. So answer politely: I don\'t know, ask them to introduce themselves.'}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 79, 'total_tokens': 171, 'completion_time': 0.205034314, 'completion_tokens_details': {'reasoning_tokens': 50}, 'prompt_time': 0.002993022, 'prompt_tokens_details': None, 'queue_time': 0.051186178, 'total_time': 0.208027336}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_70d048ba3c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--8f217325-3486-4aa8-a553-d7a447503402-0', usage_metadata={'input_tokens': 79, 'ou

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is VaibhaV and I am a Chief AI Engineer"),
        AIMessage(content="Hello VaibhaV! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='Your name is **VaibhaV**, and you work as a **Chief AI Engineer**.', additional_kwargs={'reasoning_content': 'The user asks: "Hey What\'s my name and what do I do?" The conversation: user introduced themselves as "Hi , My name is VaibhaV and I am a Chief AI Engineer". So answer: your name is VaibhaV and you are a Chief AI Engineer. Should respond accordingly.'}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 156, 'total_tokens': 248, 'completion_time': 0.19342393, 'completion_tokens_details': {'reasoning_tokens': 63}, 'prompt_time': 0.005851851, 'prompt_tokens_details': None, 'queue_time': 0.051673969, 'total_time': 0.199275781}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a28df4bce5', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--6150c49d-4210-4409-a78b-6bbca8c7d744-0', usage_metadata={'input_tokens': 156, 'output_tokens': 92, 'total_tokens': 248, 'outpu

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
!uv pip install langchain_community

Using Python 3.11.11 environment at: /Users/vaibhavarde/Desktop/AgentKrish/AgentNotes/.venv
Audited 1 package in 86ms


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [18]:
config={"configurable":{"session_id":"chat1"}}
config

{'configurable': {'session_id': 'chat1'}}

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is VaibhaV and I am a Chief AI Engineer")],
    config=config
)

In [20]:
response.content

'Hello VaibhaV! It’s great to meet you. How can I assist you today?'

In [21]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is VaibhaV and I am a Chief AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello VaibhaV! It’s great to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says: "Hi , My name is VaibhaV and I am a Chief AI Engineer". Likely they are just greeting. The assistant should respond politely. No policy issues.'}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 87, 'total_tokens': 156, 'completion_time': 0.148574634, 'completion_tokens_details': {'reasoning_tokens': 40}, 'prompt_time': 0.003219431, 'prompt_tokens_details': None, 'queue_time': 0.051090589, 'total_time': 0.151794065}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_fd1fe7f861', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--dfb9f81d-6f1a-44dd-8ae7-d960c3ac17c2-0', usage_met

In [22]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is VaibhaV.', additional_kwargs={'reasoning_content': 'User asks "What\'s my name?" They introduced themselves as VaibhaV. So answer: VaibhaV.'}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 121, 'total_tokens': 163, 'completion_time': 0.087330393, 'completion_tokens_details': {'reasoning_tokens': 25}, 'prompt_time': 0.004643322, 'prompt_tokens_details': None, 'queue_time': 0.051253518, 'total_time': 0.091973715}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_fd1fe7f861', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--ede716ed-7410-40ac-8ef8-d647e809ad55-0', usage_metadata={'input_tokens': 121, 'output_tokens': 42, 'total_tokens': 163, 'output_token_details': {'reasoning': 25}})

In [23]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is VaibhaV and I am a Chief AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello VaibhaV! It’s great to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says: "Hi , My name is VaibhaV and I am a Chief AI Engineer". Likely they are just greeting. The assistant should respond politely. No policy issues.'}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 87, 'total_tokens': 156, 'completion_time': 0.148574634, 'completion_tokens_details': {'reasoning_tokens': 40}, 'prompt_time': 0.003219431, 'prompt_tokens_details': None, 'queue_time': 0.051090589, 'total_time': 0.151794065}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_fd1fe7f861', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--dfb9f81d-6f1a-44dd-8ae7-d960c3ac17c2-0', usage_met

In [24]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I don’t have any information about your name. If you’d like, you can tell me what you’d like to be called, and I’ll use that going forward!'

In [25]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is VaibhaV and I am a Chief AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello VaibhaV! It’s great to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says: "Hi , My name is VaibhaV and I am a Chief AI Engineer". Likely they are just greeting. The assistant should respond politely. No policy issues.'}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 87, 'total_tokens': 156, 'completion_time': 0.148574634, 'completion_tokens_details': {'reasoning_tokens': 40}, 'prompt_time': 0.003219431, 'prompt_tokens_details': None, 'queue_time': 0.051090589, 'total_time': 0.151794065}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_fd1fe7f861', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--dfb9f81d-6f1a-44dd-8ae7-d960c3ac17c2-0', usage_met

In [26]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Nice to meet you, John! How can I assist you today?'

In [27]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is VaibhaV and I am a Chief AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello VaibhaV! It’s great to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says: "Hi , My name is VaibhaV and I am a Chief AI Engineer". Likely they are just greeting. The assistant should respond politely. No policy issues.'}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 87, 'total_tokens': 156, 'completion_time': 0.148574634, 'completion_tokens_details': {'reasoning_tokens': 40}, 'prompt_time': 0.003219431, 'prompt_tokens_details': None, 'queue_time': 0.051090589, 'total_time': 0.151794065}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_fd1fe7f861', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--dfb9f81d-6f1a-44dd-8ae7-d960c3ac17c2-0', usage_met

In [28]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [29]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [30]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is VaibhaV")]})

AIMessage(content='Hello VaibhaV! 👋 Nice to meet you. How can I help you today?', additional_kwargs={'reasoning_content': 'We need to respond. The user says "Hi My name is VaibhaV". Probably a greeting. We can respond politely, ask how can help. Use friendly tone.'}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 98, 'total_tokens': 164, 'completion_time': 0.136203852, 'completion_tokens_details': {'reasoning_tokens': 37}, 'prompt_time': 0.00365894, 'prompt_tokens_details': None, 'queue_time': 0.05096239, 'total_time': 0.139862792}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_e88ce9c728', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f05f9af9-e63e-4988-8b39-04ff8c1939fb-0', usage_metadata={'input_tokens': 98, 'output_tokens': 66, 'total_tokens': 164, 'output_token_details': {'reasoning': 37}})

In [31]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [32]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is VaibhaV Arde")],
    config=config
)

response

AIMessage(content='Hello VaibhaV! Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says: "Hi My name is VaibhaV Arde". Likely they are greeting and introducing themselves. The assistant should respond politely, perhaps ask how can help. No request for disallowed content. So respond friendly.'}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 100, 'total_tokens': 176, 'completion_time': 0.161337164, 'completion_tokens_details': {'reasoning_tokens': 49}, 'prompt_time': 0.003796666, 'prompt_tokens_details': None, 'queue_time': 0.049855363, 'total_time': 0.16513383}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a28df4bce5', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--525108e7-14eb-4550-9987-8ce57f7db292-0', usage_metadata={'input_tokens': 100, 'output_tokens': 76, 'total_tokens': 176, 'output_token_details': {'reasoning': 49}})

In [33]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is VaibhaV and I am a Chief AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello VaibhaV! It’s great to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says: "Hi , My name is VaibhaV and I am a Chief AI Engineer". Likely they are just greeting. The assistant should respond politely. No policy issues.'}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 87, 'total_tokens': 156, 'completion_time': 0.148574634, 'completion_tokens_details': {'reasoning_tokens': 40}, 'prompt_time': 0.003219431, 'prompt_tokens_details': None, 'queue_time': 0.051090589, 'total_time': 0.151794065}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_fd1fe7f861', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--dfb9f81d-6f1a-44dd-8ae7-d960c3ac17c2-0', usage_met

In [35]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is **VaibhaV\u202fArde**. Let me know if there’s anything else you’d like help with!'

In [36]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [37]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Vaibhav")],"language":"Hindi"})
response.content

'नमस्ते वैभव! आपका स्वागत है। मैं आपकी कैसे मदद कर सकता हूँ?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [38]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [39]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Meera")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते मीरा! आपसे मिलकर खुशी हुई। मैं आपकी कैसे मदद कर सकता/सकती हूँ?'

In [40]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [41]:
response.content

'आपका नाम मीरा है।'

In [42]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is VaibhaV and I am a Chief AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello VaibhaV! It’s great to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says: "Hi , My name is VaibhaV and I am a Chief AI Engineer". Likely they are just greeting. The assistant should respond politely. No policy issues.'}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 87, 'total_tokens': 156, 'completion_time': 0.148574634, 'completion_tokens_details': {'reasoning_tokens': 40}, 'prompt_time': 0.003219431, 'prompt_tokens_details': None, 'queue_time': 0.051090589, 'total_time': 0.151794065}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_fd1fe7f861', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--dfb9f81d-6f1a-44dd-8ae7-d960c3ac17c2-0', usage_met

In [43]:
config = {"configurable": {"session_id": "chat1"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Waht is my name?")], "language":"Marathi"},
    config=config,
)

response.content

'तुमचे नाव **वैभव** आहे.'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [44]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [47]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'I’m not sure which flavor you prefer—could you tell me your favorite ice‑cream flavor? Once I know, I can chat about it or suggest similar ones you might enjoy!'

In [48]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked about the sum\u202f2\u202f+\u202f2.'

In [49]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [50]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'I don’t have any information about your name. If you’d like to share it, feel free to let me know!'

In [51]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You haven’t actually asked a math problem yet. If you have a specific question or problem in mind, feel free to share it and I’ll be happy to help!'